In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import sys
from PIL import Image
from sklearn.cross_validation import train_test_split

In [3]:
class ImportData:
    def __init__(self,width,height):
        print("Starting...")
        labels = []
        images = np.array([])
        PATH = "dogbreed/"
        categories = np.array([])
        with open(f'{PATH}labels.csv','r') as file:
            seen = set() # set for fast O(1) amortized lookup
            next(file) # Skips the first row which are column names.
            for line in file:
                category = line.split(",")[1][:-1]
                if category in seen: continue # skip duplicate

                seen.add(category)
                categories = np.append(categories, category)
                
        with open(f'{PATH}labels.csv','r') as file:
            next(file)
            for line in file:
                label = np.array([0]*len(categories))
                category = line.split(",")[1][:-1]
                label[np.where(categories == category)] = 1
                labels.append(label)

                name = line.split(",")[0]
                img_path = f'{PATH}train/{name}.jpg'
                images = np.append(images,img_path)

        train_images, test_images, train_labels, test_labels = \
            train_test_split(images, labels, test_size=0.02, random_state=0)
        
        class train:
            def __init__(self):
                self.images = np.array([])
                self.labels = []
                self.batch_counter = 0
                
            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    batch_images.extend(self.images[:left])
                    batch_labels.extend(self.labels[:left])
                    self.batch_counter = left
                else:
                    bi = self.images[self.batch_counter:self.batch_counter+num]
                    batch_images = np.array([])
                    for index, image_path in enumerate(self.images[self.batch_counter:self.batch_counter+num]):
                        try:
                            im = Image.open(image_path)
                        except:
                            print ("Skip a corrupted file: ", image_path)
                            continue

                        im = im.resize([width,height], Image.ANTIALIAS)
                        pixels = np.asarray(im)
                        pixels = pixels/255.0
                        #print("pixels.shape:",pixels.shape)
                        #print("images.shape:",images.shape)
                        if batch_images.size == 0:
                            batch_images = pixels[None,:]
                        else:
                            batch_images = np.append(batch_images,pixels[None,:],axis=0)
                    batch_labels = self.labels[self.batch_counter:self.batch_counter+num]                  
                    self.batch_counter += num
                    
                return (batch_images, batch_labels)
                    
        class test:
            def __init__(self):
                self.images = np.array([])
                self.labels = []
                
        self.train = train()
        self.test = test()
                
        self.train.images = train_images
        self.train.labels = train_labels
        print("len(test_images):",len(test_images))
        for i, image_path in enumerate(test_images):
            #sys.stdout.write("Progress: %d%%   \r" % (round(i/len(test_images),1)) )
            #sys.stdout.flush()
            try:
                im = Image.open(image_path)
            except:
                print ("Skip a corrupted file: ", image_path)
                continue

            im = im.resize([width,height], Image.ANTIALIAS)
            pixels = np.asarray(im)
            pixels = pixels/255.0
            #print("pixels.shape:",pixels.shape)
            #print("images.shape:",images.shape)
            if self.test.images.size == 0:
                self.test.images = pixels[None,:]
            else:
                self.test.images = np.append(self.test.images,pixels[None,:],axis=0)
        self.test.labels = test_labels
        print("done!")

In [4]:
mnist = ImportData(224,224)

Starting...
len(test_images): 205
done!


In [5]:
mnist.train.images.shape

(10017,)

In [6]:
MAX_STEPS = 75
BATCH_SIZE = 128
NUM_CLASSES = 120
STEP_DISPLAY = 5
TEST_SIZE = 128

In [7]:
x = tf.placeholder(tf.float32,[None,224,224,3])
y_ = tf.placeholder(tf.float32, [None, NUM_CLASSES])

# Download and use NASNet feature vector module.
#module = hub.Module("https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/1")
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/1")

temp_images,temp_labels = mnist.train.next_batch(BATCH_SIZE)
print("temp_images.shape:",temp_images.shape)
features = module(temp_images)
logits = tf.layers.dense(features, NUM_CLASSES)
prediction = tf.nn.softmax(logits)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm

INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block1/unit_2/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block1/unit_2/bottleneck_v2/preact/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block1/unit_2/bottleneck_v2/preact/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with

INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/biases
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/weights:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block2/unit_1/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block

INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/weights:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec906

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aa

...

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_5/bottleneck_v2/preact/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_5/bottleneck_v2/preact/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec9066

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_7/bottleneck_v2/preact/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_8/bottleneck_v2/conv1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/varia

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/conv3/weights:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_9/bottleneck_v2/conv3/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_9/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block3/unit_9/bottleneck_v2/preact/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/preact/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101

INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/weights:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec906

INFO:tensorflow:Initialize variable module/resnet_v2_101/postnorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/4aab483d2ddec90660fffda73f5d14ebb0ece3a2/variables/variables' with resnet_v2_101/postnorm/moving_variance
temp_images.shape: (128, 224, 224, 3)


In [8]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(prediction), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(0.003).minimize(cross_entropy)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    test_images = mnist.test.images[:BATCH_SIZE]
    test_labels = mnist.test.labels[:BATCH_SIZE]
    
    for step in range(MAX_STEPS):
        batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)

        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        
        if (step % STEP_DISPLAY) == 0:
            print("step:",step)
            pred = sess.run(prediction,feed_dict={x: test_images})
            print("prediction:",np.argmax(pred,axis=0)[:5])
            print("y predicts:",np.argmax(mnist.test.labels,axis=0)[:5])
            print("Cost:",sess.run(cross_entropy,feed_dict={x:test_images,y_:test_labels}))
            print("Accuracy:",sess.run(accuracy, feed_dict={x: test_images, y_:test_labels}))
            print("________________________________________")
    print("Accuracy:")
    print(MAX_STEPS, sess.run(accuracy, feed_dict={x: test_images, y_:test_labels}))

step: 0
prediction: [  9 104 125 115 100]
y predicts: [ 48 107 146  76  62]
Cost: 7.4461994
Accuracy: 0.0078125
________________________________________
step: 5
prediction: [ 47  94  84  71 101]
y predicts: [ 48 107 146  76  62]
Cost: 7.008709
Accuracy: 0.0
________________________________________
step: 10
prediction: [ 57  51  39 126 101]
y predicts: [ 48 107 146  76  62]
Cost: 6.6105433
Accuracy: 0.0078125
________________________________________
step: 15
prediction: [100  60  74  98  16]
y predicts: [ 48 107 146  76  62]
Cost: 6.3151336
Accuracy: 0.03125
________________________________________
step: 20
prediction: [100 119  68 118  16]
y predicts: [ 48 107 146  76  62]
Cost: 6.194519
Accuracy: 0.0234375
________________________________________
step: 25
prediction: [113 111  68  80  88]
y predicts: [ 48 107 146  76  62]
Cost: 6.1229043
Accuracy: 0.0
________________________________________
step: 30
prediction: [76 23 65 40 88]
y predicts: [ 48 107 146  76  62]
Cost: 6.1425753
Accura